In [107]:
import torch
from torch import nn
import pandas as pd
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split

def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    dataNorm["CreditLevel"]=dataset["CreditLevel"]-1
    return dataNorm

# data preprocessing
train_dataset = pd.read_csv("BankChurners.csv")
print(f"Original data frame has {train_dataset.shape[0]} rows and {train_dataset.shape[1]} columns.")
train_dataset = train_dataset.drop(['CustomerId','Geography'],axis=1)
train_dataset = normalize(train_dataset)
train_dataset.info()
print(f"preprocessing data frame has {train_dataset.shape[0]} rows and {train_dataset.shape[1]} columns.")
train_dataset.head()


Original data frame has 9000 rows and 10 columns.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tenure           9000 non-null   float64
 1   Balance          9000 non-null   float64
 2   NumOfProducts    9000 non-null   float64
 3   HasCrCard        9000 non-null   float64
 4   IsActiveMember   9000 non-null   float64
 5   EstimatedSalary  9000 non-null   float64
 6   Exited           9000 non-null   float64
 7   CreditLevel      9000 non-null   int64  
dtypes: float64(7), int64(1)
memory usage: 562.6 KB
preprocessing data frame has 9000 rows and 8 columns.


,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,CreditLevel
0,0.3,0.484985,0.000000,1.0,0.0,0.643290,1.0,7
1,0.6,0.000000,0.000000,1.0,0.0,0.251062,1.0,6
2,0.2,0.728934,0.000000,1.0,0.0,0.015250,0.0,6
3,0.2,0.407651,0.333333,1.0,0.0,0.449146,0.0,1
4,0.7,0.435819,0.333333,1.0,0.0,0.513377,0.0,6


In [108]:
x_train = torch.tensor(train_dataset.drop('CreditLevel',axis=1).values.astype('float32'))
y_train = torch.tensor(train_dataset['CreditLevel'].values)
train_tensor = TensorDataset(x_train,y_train)
n_train = int(len(train_tensor)*0.8)
n_valid = len(train_tensor) - n_train
ds_train,ds_valid = random_split(train_tensor,[n_train,n_valid])
dl_train = DataLoader(ds_train,batch_size = 100,shuffle=True)
dl_valid_batch_size = 30
dl_valid = DataLoader(ds_valid,batch_size = dl_valid_batch_size,shuffle=False)

for features,labels in dl_train:
    print(features,labels)
    break

tensor([[0.7000, 0.2607, 0.3333, 0.0000, 0.0000, 0.9133, 1.0000],
        [0.5000, 0.5128, 0.0000, 1.0000, 0.0000, 0.6379, 0.0000],
        [0.3000, 0.2367, 0.3333, 1.0000, 1.0000, 0.5277, 0.0000],
        [0.7000, 0.3055, 0.3333, 1.0000, 1.0000, 0.9275, 0.0000],
        [0.4000, 0.3446, 0.0000, 1.0000, 1.0000, 0.7075, 0.0000],
        [0.8000, 0.6809, 0.3333, 0.0000, 0.0000, 0.5493, 0.0000],
        [1.0000, 0.2937, 0.0000, 1.0000, 1.0000, 0.8608, 0.0000],
        [0.5000, 0.5580, 0.0000, 0.0000, 1.0000, 0.1585, 0.0000],
        [1.0000, 0.0000, 0.3333, 1.0000, 1.0000, 0.4752, 0.0000],
        [0.2000, 0.3172, 0.0000, 0.0000, 0.0000, 0.0202, 0.0000],
        [0.7000, 0.0000, 0.0000, 1.0000, 0.0000, 0.4448, 1.0000],
        [0.7000, 0.7048, 0.6667, 1.0000, 0.0000, 0.8327, 1.0000],
        [0.7000, 0.3979, 0.3333, 1.0000, 0.0000, 0.4513, 1.0000],
        [0.9000, 0.4005, 0.0000, 1.0000, 0.0000, 0.6205, 0.0000],
        [0.0000, 0.7409, 0.0000, 1.0000, 1.0000, 0.2318, 1.0000],
        [0

In [109]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import kaiming_uniform_,xavier_uniform_

class Net(nn.Module):
    def __init__(self,n_inputs):
        super(Net, self).__init__()
        # input to first hidden layer
        self.hidden1 = nn.Linear(n_inputs, 14)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.dropout1 = nn.Dropout(0.3)
        self.act1 = nn.ReLU()
        
        # second hidden layer
        # self.hidden2 = nn.Linear(14, 10)
        # kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        # self.dropout2 = nn.Dropout(0.3)
        # self.act2 = nn.ReLU()
        
        # third hidden layer and output
        self.hidden3 = nn.Linear(14, 10)
        xavier_uniform_(self.hidden3.weight)
        #self.dropout3 = nn.Dropout(0.3)
        self.act3 = nn.Softmax(dim=1)
    
    def forward(self,X):
         # input to first hidden layer
        X = self.hidden1(X)
        X = self.dropout1(X)
        X = self.act1(X)
        
        # second hidden layer
        #X = self.hidden2(X)
        #X = self.dropout2(X)
        #X = self.act2(X)
        
        # output layer
        X = self.hidden3(X)
        #X = self.dropout3(X)
        X = self.act3(X)
        return X

model = Net(7)
print(model)

Net(
  (hidden1): Linear(in_features=7, out_features=14, bias=True)
  (dropout1): Dropout(p=0.3, inplace=False)
  (act1): ReLU()
  (hidden3): Linear(in_features=14, out_features=10, bias=True)
  (act3): Softmax(dim=1)
)


In [110]:
#define a Loss function and optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [111]:
# Train the network
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs,targets) in enumerate(dl_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,     1] loss: 0.023
[2,     1] loss: 0.023
[3,     1] loss: 0.023
[4,     1] loss: 0.023
[5,     1] loss: 0.023
[6,     1] loss: 0.023
[7,     1] loss: 0.023
[8,     1] loss: 0.023
[9,     1] loss: 0.023
[10,     1] loss: 0.023
[11,     1] loss: 0.023
[12,     1] loss: 0.023
[13,     1] loss: 0.023
[14,     1] loss: 0.023
[15,     1] loss: 0.023
[16,     1] loss: 0.023
[17,     1] loss: 0.023
[18,     1] loss: 0.022
[19,     1] loss: 0.023
[20,     1] loss: 0.023


In [112]:
# model test
model.eval()
accuracy = 0.0
total = 0.0
with torch.no_grad():
    for data in dl_valid:
        inputs,targets = data
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += targets.size(0)
        accuracy += (predicted == targets).sum().item()

accuracy = (100*accuracy/total)
print(accuracy)

20.666666666666668


In [114]:
def testClassess():
    number_of_labels = 10
    class_correct = list(0. for i in range(number_of_labels))
    class_total = list(0. for i in range(number_of_labels))
    with torch.no_grad():
        for data in dl_valid:
            inputs,targets = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == targets).squeeze()
            for i in range(dl_valid_batch_size):
                target = targets[i]
                class_correct[target] += c[i].item()
                class_total[target] += 1

    for i in range(number_of_labels):
        print('Accuracy of %5d : %2d %%' % (
            i, 100 * class_correct[i] / class_total[i]))
        
        
testClassess()

Accuracy of     0 :  0 %
Accuracy of     1 :  0 %
Accuracy of     2 :  0 %
Accuracy of     3 :  0 %
Accuracy of     4 :  0 %
Accuracy of     5 :  0 %
Accuracy of     6 : 99 %
Accuracy of     7 :  0 %
Accuracy of     8 :  0 %
Accuracy of     9 :  0 %
